In [1]:
import os
os.environ["TF_USE_LEGACY_KERAS"] = "1"
import pprint
import tempfile

from typing import Dict, Text

import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs

2025-04-28 14:11:11.997868: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745863872.052492   11087 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745863872.064825   11087 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1745863872.096892   11087 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1745863872.096925   11087 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1745863872.096927   11087 computation_placer.cc:177] computation placer alr

## Data Transformation

In [ ]:
# Import the Movielens dataset.
ratings = tfds.load("movielens/100k-ratings", split="train")
movies = tfds.load("movielens/100k-movies", split="train")

# View the dataset columns.
pprint.pprint(ratings.element_spec)

{'bucketized_user_age': TensorSpec(shape=(), dtype=tf.float32, name=None),
 'movie_genres': TensorSpec(shape=(None,), dtype=tf.int64, name=None),
 'movie_id': TensorSpec(shape=(), dtype=tf.string, name=None),
 'movie_title': TensorSpec(shape=(), dtype=tf.string, name=None),
 'raw_user_age': TensorSpec(shape=(), dtype=tf.float32, name=None),
 'timestamp': TensorSpec(shape=(), dtype=tf.int64, name=None),
 'user_gender': TensorSpec(shape=(), dtype=tf.bool, name=None),
 'user_id': TensorSpec(shape=(), dtype=tf.string, name=None),
 'user_occupation_label': TensorSpec(shape=(), dtype=tf.int64, name=None),
 'user_occupation_text': TensorSpec(shape=(), dtype=tf.string, name=None),
 'user_rating': TensorSpec(shape=(), dtype=tf.float32, name=None),
 'user_zip_code': TensorSpec(shape=(), dtype=tf.string, name=None)}


In [23]:
# Prepare the ratings dataset for processing
ratings = ratings.map(lambda x: {
    'user_id': x['user_id'],
    'movie_id': x['movie_id'],
    'user_rating': x['user_rating'],
    'timestamp': x['timestamp'],
    'movie_genres': x['movie_genres'],
    'bucketized_user_age': x['bucketized_user_age'],
    'user_occupation_label': x['user_occupation_label']
})

# Convert categorical features 

# Set up a large number of bins to reduce the chance of hash collisions.
NUM_HASHING_BINS = 200_000

movie_title_hashing = tf.keras.layers.Hashing(
    num_bins=NUM_HASHING_BINS
)

# Conver to embedding
movie_title_embedding = tf.keras.layers.Embedding(
    # Let's use the explicit vocabulary lookup.
    input_dim=NUM_HASHING_BINS,
    output_dim=32
)


In [25]:
movie_title_model = tf.keras.Sequential([movie_title_lookup, movie_title_embedding])

In [26]:
movie_title_model(["Star Wars (1977)"])

<tf.Tensor: shape=(1, 32), dtype=float32, numpy=
array([[ 0.03373393,  0.00759703,  0.04229622, -0.04404191,  0.03500244,
        -0.02696208, -0.01288205,  0.00450683,  0.02644501,  0.02544909,
         0.02573315,  0.02732192, -0.00416629, -0.03338205,  0.00879828,
         0.04101123,  0.02224832,  0.04004196,  0.04615921,  0.04891871,
         0.02474656, -0.04375819, -0.04963802, -0.00394917,  0.00650693,
        -0.01145471,  0.03538618,  0.04382434, -0.02577845,  0.0140728 ,
         0.01044954,  0.04170798]], dtype=float32)>

In [ ]:
user_id_lookup = tf.keras.layers.StringLookup()
user_id_lookup.adapt(ratings.map(lambda x: x["user_id"]))

user_id_embedding = tf.keras.layers.Embedding(user_id_lookup.vocabulary_size(), 32)

user_id_model = tf.keras.Sequential([user_id_lookup, user_id_embedding])

In [ ]:
for x in ratings.take(3).as_numpy_iterator():
  print(f"Timestamp: {x['timestamp']}.")